In [210]:
import os 
import pandas as pd

def create_plateau(n, df):
    plateau_counts = []
    max_val = float(df.max())
    print("Max: ",max_val)
    min_val = float(df.min())
    print("Min: ",min_val)
    plateau_list = []
    plateau_size = (max_val-min_val)/n
    print("Plateua Size: ",plateau_size)
    current_plateau = min_val
    for i in range(0,n):
        new_plateau = current_plateau + plateau_size
        print(str(current_plateau) + " --- " + str(new_plateau))
        df_filter = int(df[(df["BUS"] > current_plateau) & (df["BUS"] < new_plateau)].count()[0])
        current_plateau = new_plateau
        print(df_filter)
        plateau_counts.append(df_filter)
    return plateau_counts

def only_max(df, tolerance):
    max_val = float(df.max())
    plateau_count = df[df["BUS"] > (max_val-tolerance)].count()[0]
    return plateau_count


def load_data(path, plateau_size=None):
    no_dash_list = []
    dash_list = []

    for dir in os.listdir(path):
        for subdir in os.listdir(os.path.join(path,dir)):
            load_path = os.path.join(path, dir, subdir)
            df = pd.read_csv(load_path)
            df = df[["BUS"]]
            if plateau_size is None:
                plateau_counts = only_max(df,tolerance=20)
            else:
                plateau_counts = create_plateau(plateau_size, df)
            if "no" in subdir:
                no_dash_list.append(plateau_counts)
            else:
                dash_list.append(plateau_counts)
    NoDash = pd.DataFrame(no_dash_list)
    NoDash["Dash"] = "no"
    Dash = pd.DataFrame(dash_list)
    Dash["Dash"] = "yes"
    df = pd.concat((NoDash,Dash))
    return df.sample(frac=1).reset_index(drop=True)
    

In [211]:
df = load_data(path = "Printer3D")
df

,0,Dash
0,341,no
1,586,yes
2,987,no
3,1553,yes
4,568,yes
5,1169,no


In [212]:
df.to_csv("3d_printer_plateau.csv", index=False)

In [213]:
df = pd.read_csv("3d_printer_plateau.csv")
df

,0,Dash
0,341,no
1,586,yes
2,987,no
3,1553,yes
4,568,yes
5,1169,no


In [220]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import cross_val_score 
y = df[["Dash"]]
X = df.drop("Dash", axis=1)
dt = DecisionTreeClassifier()
cv_results = cross_val_score(dt, X, y, cv=3)
print(cv_results)

[0.5 0.5 1. ]
